# Validating simulation experiments (SED-ML files)

This tutorial illustrates how to validate simulation experiments encoded in the [Simulation Experiment Description Markup Language (SED-ML)](http://sed-ml.org/). This format uses [URNs](https://sed-ml.org/urns.html) to describe the language of each model. This format uses the [Kinetic Simulation Algorithm Ontology (KiSAO)](https://github.com/SED-ML/KiSAO) to describe algorithms and algorithm parameters.

<div class="alert alert-block alert-info">
    The methods described here do not validate the sources of the models (e.g., CellML, SBML files) of SED-ML files or targets for changes to models or for outputs of simulations of models. More comprehensive validation is available through validation of entire simulation projects (COMBINE/OMEX archives). This encompasses validation of the sources of models in SED-ML files and targets for models for model changes and simulation observables. Please see the <a href="4.%20Validating%20simulation%20projects%20(COMBINE-OMEX%20archives).ipynb">simulation project validation tutorial</a> for more information.
</div>

<div class="alert alert-block alert-info">
    SED-ML is under active development. The SED-ML community is working toward a single interpretation of SED-ML. BioSimulators supports the interpretation of SED-ML described at <a href="https://docs.biosimulations.org/concepts/conventions/simulation-experiments/">https://docs.biosimulations.org</a>.
</div>

## 1. Validate a SED-ML file online

The easiest way to validate a SED-ML file is to use the web interface at https://run.biosimulations.org. An HTTP API for validating SED-ML files is also available at [https://combine.api.biosimulations.org](https://combine.api.biosimulations.org/).

## 2. Validate a SED-ML file with the BioSimulators command-line application

First, install [BioSimulators-utils](https://github.com/biosimulators/Biosimulators_utils). Installation instructions are available at [https://docs.biosimulators.org](https://docs.biosimulators.org/Biosimulators_utils). Note, BioSimulators-utils must be installed with the installation options for the model languages that you wish to validate. A Docker image with BioSimulators utils and all dependencies is also available ([`ghcr.io/biosimulators/biosimulators`](https://github.com/biosimulators/Biosimulators/pkgs/container/biosimulators)).

Inline help for the `biosimulators-utils` command-line program is available by running the program with the `--help` option.

In [1]:
!biosimulators-utils --help

Invalid -W option ignored: invalid module name: 'biosimulators_utils.warnings'
usage: biosimulators-utils [-h] [-d] [-q] [-v]
                           {convert,exec,validate-project,validate-metadata,validate-simulation,validate-model,build-project}
                           ...

Utilities for working with containerized biosimulation tools

optional arguments:
  -h, --help            show this help message and exit
  -d, --debug           full application debug mode
  -q, --quiet           suppress all console output
  -v, --version         show program's version number and exit

sub-commands:
  {convert,exec,validate-project,validate-metadata,validate-simulation,validate-model,build-project}
    convert             Convert files among formats
    exec                Execute a model project (COMBINE/OMEX archive)
    validate-project    Validate a model project (COMBINE/OMEX archive)
    validate-metadata   Validate metadata (OMEX Metadata file)
    validate-simulation
             

In [2]:
!biosimulators-utils validate-simulation --help

Invalid -W option ignored: invalid module name: 'biosimulators_utils.warnings'
usage: biosimulators-utils validate-simulation [-h] filename

Validate a simulation experiment (SED-ML file)

positional arguments:
  filename    Path to a SED-ML file

optional arguments:
  -h, --help  show this help message and exit


Next, use the command-line program to validate the [simulation experiment](../_data/Ciliberto-J-Cell-Biol-2003-morphogenesis-checkpoint-continuous.sedml).

In [3]:
!biosimulators-utils validate-simulation ../_data/Ciliberto-J-Cell-Biol-2003-morphogenesis-checkpoint-continuous.sedml

Invalid -W option ignored: invalid module name: 'biosimulators_utils.warnings'


If the simulation is invalid, a list of errors will be printed to your console.

## 3. Validate a SED-ML file programmatically with Python

First, install [BioSimulators-utils](https://github.com/biosimulators/Biosimulators_utils). Installation instructions are available at [https://docs.biosimulators.org](https://docs.biosimulators.org/Biosimulators_utils). Note, BioSimulators-utils must be installed with the installation options for the model languages that you wish to validate. A Docker image with BioSimulators utils and all dependencies is also available ([`ghcr.io/biosimulators/biosimulators`](https://github.com/biosimulators/Biosimulators/pkgs/container/biosimulators)).

Next, import BioSimulators-utils' SED-ML reader.

In [4]:
from biosimulators_utils.sedml.io import SedmlSimulationReader

Next, use this class to read the SED-ML file. To skip validating the sources of models and targets to them, set the `validate_models_with_language` argument to `False`.

In [5]:
reader = SedmlSimulationReader()
simulation_filename = '../_data/Ciliberto-J-Cell-Biol-2003-morphogenesis-checkpoint-continuous.sedml'
sed_doc = reader.run(simulation_filename, validate_models_with_languages=False, validate_targets_with_model_sources=False)

The `run` method raises exceptions on invalid SED-ML files. The `run` method also stores lists of errors and warnings in the instance of `SedmlSimulationReader`. This information can be accessed by catching the exceptions raised by the `run` method. Human-readable messages can then be printed out using the `flatten_nested_list_of_strings` method.

In [6]:
try:
    sed_doc = reader.run(simulation_filename, validate_models_with_languages=False, validate_targets_with_model_sources=False)
except (ValueError, NotImplementedError):
    pass

In [7]:
from biosimulators_utils.utils.core import flatten_nested_list_of_strings
from warnings import warn

if reader.warnings:
    warn(flatten_nested_list_of_strings(reader.warnings), UserWarning)

if reader.errors:
    raise ValueError(flatten_nested_list_of_strings(reader.errors))

The output of the `run` method (`sed_doc`) is a data structure that represents all of the information contained in the SED-ML file, including the models, simulations, tasks, reports, and plots. This data structure is an instance of `biosimulators_utils.sedml.data_model.SedDocument`. The `vars` method and `inspect` module can be used to inspect the attributes and methods of this data structure.

In [8]:
vars(sed_doc)

{'level': 1,
 'version': 3,
 'models': [<biosimulators_utils.sedml.data_model.Model at 0x7f9db42d1a60>],
 'simulations': [<biosimulators_utils.sedml.data_model.UniformTimeCourseSimulation at 0x7f9db42d1d90>],
 'tasks': [<biosimulators_utils.sedml.data_model.Task at 0x7f9db42d1940>],
 'data_generators': [<biosimulators_utils.sedml.data_model.DataGenerator at 0x7f9db42d1700>,
 'outputs': [<biosimulators_utils.sedml.data_model.Report at 0x7f9db42d1610>,
 'metadata': None}

In [9]:
import inspect
dict(inspect.getmembers(sed_doc, predicate=inspect.ismethod))

{'__init__': <bound method SedDocument.__init__ of <biosimulators_utils.sedml.data_model.SedDocument object at 0x7f9db42d1eb0>>,
 'is_equal': <bound method SedDocument.is_equal of <biosimulators_utils.sedml.data_model.SedDocument object at 0x7f9db42d1eb0>>,
 'to_tuple': <bound method SedDocument.to_tuple of <biosimulators_utils.sedml.data_model.SedDocument object at 0x7f9db42d1eb0>>}